# SST Biases


In [1]:
%matplotlib inline

import pandas as pd
import intake
catalog = intake.cat.access_nri
import matplotlib.pyplot as plt
import xarray as xr
import numpy as np
import IPython.display
import cmocean as cm
import cartopy.crs as ccrs
import cartopy.feature as cft
from dask.distributed import Client, progress


import sys, os

os.chdir(os.path.join(os.getcwd(), "ACCESS-OM2-updated/figures/temperature_biases"))
sys.path.append(os.path.join(os.getcwd(), '..'))  # so we can import ../exptdata
import exptdata
print('Available exptdata keys: ', [k for k in exptdata.exptdict.keys()])

Available exptdata keys:  ['1deg', '025deg', '01deg']


In [2]:
# use common start and end dates for all runs
tstart = exptdata.clim_tstart
tend = exptdata.clim_tend

In [3]:
figdir = ''
def savefigure(fname):
    plt.savefig(os.path.join(figdir, fname+'.png'),dpi=300, bbox_inches="tight")  # comment out to disable saving
    plt.savefig(os.path.join(figdir, fname+'.pdf'),dpi=300, bbox_inches="tight")  # comment out to disable saving
    return

In [4]:
client = Client("tcp://10.6.43.39:8786")
client

<Client: 'tcp://10.6.46.29:8786' processes=2 threads=48, memory=188.56 GiB>

In [7]:
ekey='1deg'
expt = exptdata.exptdict[ekey]['expt']
n_files = exptdata.exptdict[ekey]['n_files']
time_units = exptdata.exptdict[ekey]['time_units']
offset = exptdata.exptdict[ekey]['offset']

cat_subset = catalog[expt]
var_search = cat_subset.search(variable='temp', frequency='1 monthly')
darray = var_search.to_dask()
darray = darray['temp']
if offset:
    td = pd.Timedelta(days=offset)
    darray = darray.assign_coords(time=darray.time + td)
surface_temp_10 = darray.isel(st_ocean=0).sel(time=slice(tstart,tend)).mean('time')
cat_subset = catalog['woa13/10_KDS50']
var_search = cat_subset.search(variable='temp')
darray = var_search.to_dask()
darray = darray['temp']
temp_WOA13_10 = darray.isel(ZT=0).mean('time')
#temp_WOA13_10 = cc.get_nc_variable('woa13/10_KDS50', 'ocean_temp_salt.res.nc', 'temp').isel(ZT=0).mean('time')

temp_anom_10 = surface_temp_10 - temp_WOA13_10.values - 273.15
temp_anom_10.load()

IPython.display.clear_output()

temp_anom_10.plot()

VariableNotFoundError: No files were found containing 'temp' in the 'woa13/10_KDS50' experiment

In [9]:
ekey='025deg'
expt = exptdata.exptdict[ekey]['expt']
n_files = exptdata.exptdict[ekey]['n_files']
time_units = exptdata.exptdict[ekey]['time_units']
offset = exptdata.exptdict[ekey]['offset']
cat_subset = catalog[expt]
var_search = cat_subset.search(variable='temp', frequency='1 monthly')
darray = var_search.to_dask()
darray = darray['temp']
if offset:
    td = pd.Timedelta(days=offset)
    darray = darray.assign_coords(time=darray.time + td)
surface_temp_025 = darray.isel(st_ocean=0).sel(time=slice(tstart,tend)).mean('time')
cat_subset = catalog['woa13/025_KDS50']
var_search = cat_subset.search(variable='temp')
darray = var_search.to_dask()
darray = darray['temp']
temp_WOA13_025 = darray.isel(ZT=0).mean('time')
temp_anom_025 = surface_temp_025 - temp_WOA13_025.values - 273.15

temp_anom_025.load()

IPython.display.clear_output()

temp_anom_025.plot()

VariableNotFoundError: No files were found containing 'temp' in the 'woa13/025_KDS50' experiment

In [10]:
ekey='01deg'
expt = exptdata.exptdict[ekey]['expt']
n_files = exptdata.exptdict[ekey]['n_files']
time_units = exptdata.exptdict[ekey]['time_units']
offset = exptdata.exptdict[ekey]['offset']
cat_subset = catalog[expt]
var_search = cat_subset.search(variable='temp')
darray = var_search.to_dask()
darray = darray['temp']
if offset:
    td = pd.Timedelta(days=offset)
    darray = darray.assign_coords(time=darray.time + td)
surface_temp_01 = darray.isel(st_ocean=0).sel(time=slice(tstart,tend)).mean('time')
cat_subset = catalog['woa13/01']
var_search = cat_subset.search(variable='temp')
darray = var_search.to_dask()
darray = darray['temp']
temp_WOA13_01 = darray.isel(ZT=0).mean('time')
temp_anom_01 = surface_temp_01 - temp_WOA13_01.values - 273.15
temp_anom_01.load()
IPython.display.clear_output()

temp_anom_01.plot()

VariableNotFoundError: No files were found containing 'temp' in the 'woa13/01' experiment

In [ ]:
plt.figure(figsize=(14,8))
clev = np.arange(-3,3.01,0.3)
land_50m = cft.NaturalEarthFeature('physical', 'land', '50m',
                                        edgecolor='black',
                                        facecolor='gray',linewidth=0.5)

ax1 = plt.subplot(2,2,1,projection=ccrs.Robinson(central_longitude=-100))
ax1.coastlines(resolution='50m')
ax1.add_feature(land_50m)
p1 = temp_anom_10.plot.contourf(cmap=cm.cm.balance,levels=clev,add_colorbar=False,transform=ccrs.PlateCarree())
plt.title('(a) ACCESS-OM2')

ax2 = plt.subplot(2,2,2,projection=ccrs.Robinson(central_longitude=-100))
ax2.coastlines(resolution='50m')
ax2.add_feature(land_50m)
p1 = temp_anom_025.plot.contourf(cmap=cm.cm.balance,levels=clev,add_colorbar=False,transform=ccrs.PlateCarree())
plt.title('(b) ACCESS-OM2-025')

ax3 = plt.subplot(2,2,3,projection=ccrs.Robinson(central_longitude=-100))
ax3.coastlines(resolution='50m')
ax3.add_feature(land_50m)
p1 = temp_anom_01.plot.contourf(cmap=cm.cm.balance,levels=clev,add_colorbar=False,transform=ccrs.PlateCarree())
plt.title('(c) ACCESS-OM2-01')

ax4 = plt.subplot(2,2,4,projection=ccrs.Robinson(central_longitude=-100))
ax4.coastlines(resolution='50m')
ax4.add_feature(land_50m)
p4 = temp_WOA13_01.plot.contourf(cmap=cm.cm.thermal,levels=np.arange(-2,30.01,1),add_colorbar=False,transform=ccrs.PlateCarree())
plt.title('(d) WOA13')

ax5 = plt.axes([0.92,0.52,0.01,0.33])
cb = plt.colorbar(p1,cax=ax5,orientation='vertical')
cb.ax.set_ylabel('SST anomaly (°C)')

ax6 = plt.axes([0.92,0.13,0.01,0.33])
cb = plt.colorbar(p4,cax=ax6,orientation='vertical')
cb.ax.set_ylabel('SST (°C)')

savefigure('global_sst_bias')